In [1]:
# Load packages
#import tensorflow as tf
#from tensorflow import keras
import pickle 
import numpy as np
import pandas as pd
import os
import glob
import scipy as scp
import scipy.stats as scps
from scipy.optimize import differential_evolution
from scipy.optimize import minimize
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from dataset_generator import bin_arbitrary_fptd

# Load my own functions
#import dnnregressor_train_eval_keras as dnnk
#import make_data_wfpt as mdw
#from kde_training_utilities import kde_load_data
import ddm_data_simulation as ddm_sim

In [2]:
# Get Base files
dir_ = '/media/data_cifs/afengler/data/michael_bg_stn/'
file_list = glob.glob(dir_ + 'loconf*')
file_list.extend(glob.glob(dir_ + 'hiconf*'))

In [72]:
# Make full dataset and write to csv
pd_list = []

for file_ in file_list:
    pd_tmp = pd.read_csv(file_, sep = '\t')
    pd_tmp['file'] = file_
    pd_list.append(pd_tmp)
data = pd.concat(pd_list)

# Preprocess
data['rt'] = (data['avg_cycles'] * 10) / 1000
data.loc[data['cnt_err'] == 0, 'cnt_err'] = - 1
data['choice'] = data['cnt_err']
# Write to file
data.to_csv(dir_ + 'bg_stn_full.csv', index = False)

In [3]:
# Read from file if desired
# data = pd.read_csv('/media/data_cifs/afengler/data/michael_bg_stn/bg_stn_full.csv')
data = pd.read_csv(dir_ + 'bg_stn_full.csv')

In [19]:
# Make data array
files_ = np.unique(data['file'])
data_array = np.zeros((len(files_), int(data.shape[0] / len(files_)), 2))
cnt = 0
step = 5000
for file_ in files_:
    data_array[cnt] = data[data['file'] == file_].iloc[:, -2:]
    cnt += 1
    
# pickle.dump((data_array, list(files_)), open('/media/data_cifs/afengler/data/real/bg_stn_sampling_ready.pickle', 'wb'))


In [25]:
# Make binned data
data_array = pickle.load(open('/media/data_cifs/afengler/data/real/bg_stn_sampling_ready.pickle', 'rb'))
binned_data = np.zeros((data_array[0].shape[0], 256, 2))
for i in range(binned_data.shape[0]):
    data_tmp = data_array[0][i].copy()
    data_tmp[:, 0] = data_tmp[:, 0]
    binned_data[i] = bin_arbitrary_fptd(out = data_tmp,
                                        max_t = 10)
    
pickle.dump((binned_data, data_array[1]), 
            open('/media/data_cifs/afengler/data/real/bg_stn_binned.pickle', 'wb'), 
            protocol = 2)

[ 0.          0.03921569  0.07843137  0.11764706  0.15686275  0.19607843
  0.23529412  0.2745098   0.31372549  0.35294118  0.39215686  0.43137255
  0.47058824  0.50980392  0.54901961  0.58823529  0.62745098  0.66666667
  0.70588235  0.74509804  0.78431373  0.82352941  0.8627451   0.90196078
  0.94117647  0.98039216  1.01960784  1.05882353  1.09803922  1.1372549
  1.17647059  1.21568627  1.25490196  1.29411765  1.33333333  1.37254902
  1.41176471  1.45098039  1.49019608  1.52941176  1.56862745  1.60784314
  1.64705882  1.68627451  1.7254902   1.76470588  1.80392157  1.84313725
  1.88235294  1.92156863  1.96078431  2.          2.03921569  2.07843137
  2.11764706  2.15686275  2.19607843  2.23529412  2.2745098   2.31372549
  2.35294118  2.39215686  2.43137255  2.47058824  2.50980392  2.54901961
  2.58823529  2.62745098  2.66666667  2.70588235  2.74509804  2.78431373
  2.82352941  2.8627451   2.90196078  2.94117647  2.98039216  3.01960784
  3.05882353  3.09803922  3.1372549   3.17647059  3.

In [ ]:
# Unused
for i in zip(data_arr,['random' for i in range(data_arr.shape[0])]):
    print(i)

In [26]:
binned_data_test = pickle.load(open('/media/data_cifs/afengler/data/real/bg_stn_binned.pickle', 'rb'))

In [11]:
binned_data.shape

(6, 256, 2)

In [16]:
binned_data_test[0][0].shape

(256, 2)

In [27]:
np.sum(binned_data[0] == binned_data_test[0][0])

512

In [22]:
data_array.shape

(6, 5000, 2)

In [24]:
data_array[1]

['/media/data_cifs/afengler/data/michael_bg_stn/hiconf_go_stnhi.txt',
 '/media/data_cifs/afengler/data/michael_bg_stn/hiconf_go_stnlo.txt',
 '/media/data_cifs/afengler/data/michael_bg_stn/hiconf_go_stnmid.txt',
 '/media/data_cifs/afengler/data/michael_bg_stn/loconf7_go_stnhi.txt',
 '/media/data_cifs/afengler/data/michael_bg_stn/loconf7_go_stnlo.txt',
 '/media/data_cifs/afengler/data/michael_bg_stn/loconf7_go_stnmid.txt']